In [5]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc, precision_score, recall_score, f1_score, accuracy_score, RocCurveDisplay
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
REQUIRED_COL_BASE = [
    'label',
    'global_session_time',

]

In [6]:
def generate_roc_curve(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_stats = {
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'thresholds': thresholds.tolist(),
        'auc': float(auc(fpr, tpr))
    }
    
    return roc_stats


def eval_stats(y_true, y_scores, model):
    y_scores = np.where(y_scores > 0.5, 1, 0)
    return {
        'model': model,
        f'acc': accuracy_score(y_true, y_scores),
        f'precision': precision_score(y_true, y_scores),
        f'recall': recall_score(y_true, y_scores),
        f'f1': f1_score(y_true, y_scores)
}
    
    
def results(df):
    roc_matrix = {}
    acccuracy_metrics = []
    for col in [col for col in df.columns if 'seq' in col]:
        print(f'Generating ROC curve for {col}')
        roc_matrix[col] = generate_roc_curve(df['label'].to_array(), df[col].to_array())
        print(f'Generating accuracy metrics for {col}')
        acccuracy_metrics.append(eval_stats(df['label'].to_array(), df[col].to_array(), col))
    
    return roc_matrix, pd.DataFrame.from_records(acccuracy_metrics)


def get_auc_from_json(json):
    auc = []
    with open(json) as f:
        data = json.load(f)

    for k, v in data.items():
        for i, j in v.items():
            if i == 'auc':
                auc.append(j)
    return auc
    

In [7]:
df = cudf.read_parquet('rl_ready_data/files_used_30/predicted_data.parquet')

In [8]:
eval_thresh, test_thresh = (
    (int(df.shape[0] * .7), int(df.shape[0] * .85)),
    (int(df.shape[0] * .85), int(df.shape[0]))
)

NameError: name 'df' is not defined

In [72]:
df = df.rename(columns={'continue_work_session_30_minutes': 'label'})

In [74]:
cols = REQUIRED_COL_BASE + [col for col in df.columns if 'seq' in col]
df = df[cols].to_pandas()
eval_df, test_df = df.iloc[eval_thresh[0]:eval_thresh[1]], df.iloc[test_thresh[0]:test_thresh[1]]


In [ ]:
roc_matrix_eval, acccuracy_metrics_eval = results(eval_df)

In [ ]:
roc_matrix_test, accuracy_metrics_test = results(test_df)

In [1]:
import pandas as pd
import json

In [2]:
df_eval, df_test = (
    pd.read_csv('lstm_eval_stats_files_30.csv'),
    pd.read_csv('lstm_test_stats_files_30.csv')
)

In [ ]:
auc_list_eval, auc_list_test = (
    get_auc_from_json('roc_matrix_evaluation_30.json'),
    get_auc_from_json('roc_matrix_test_30.json')
)

In [35]:
df_eval['auc'] = auc_list
df_test['auc'] = auc_list_test


df_eval['model'] = df_eval['model'].apply(lambda x: x.replace('_', ' '))
df_test['model'] = df_test['model'].apply(lambda x: x.replace('_', ' '))

In [34]:
print(df_eval.round(3).to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrr}
\toprule
model & acc & precision & recall & f1 & auc \\
\midrule
seq 1 & 0.691 & 0.684 & 0.568 & 0.621 & 0.740 \\
seq 10 & 0.691 & 0.680 & 0.579 & 0.626 & 0.742 \\
seq 20 & 0.692 & 0.683 & 0.577 & 0.625 & 0.742 \\
seq 30 & 0.689 & 0.687 & 0.554 & 0.614 & 0.741 \\
seq 30 heuristic & 0.689 & 0.688 & 0.554 & 0.614 & 0.740 \\
seq 40 & 0.690 & 0.696 & 0.539 & 0.608 & 0.743 \\
\bottomrule
\end{tabular}



In [36]:
print(df_test.to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrr}
\toprule
model & acc & precision & recall & f1 & auc \\
\midrule
seq 1 & 0.691 & 0.698 & 0.587 & 0.637 & 0.747 \\
seq 10 & 0.692 & 0.694 & 0.596 & 0.641 & 0.744 \\
seq 20 & 0.693 & 0.696 & 0.596 & 0.642 & 0.749 \\
seq 30 & 0.691 & 0.704 & 0.574 & 0.632 & 0.747 \\
seq 30 heuristic & 0.691 & 0.704 & 0.574 & 0.632 & 0.746 \\
seq 40 & 0.690 & 0.713 & 0.550 & 0.621 & 0.750 \\
\bottomrule
\end{tabular}

